In [1]:
%pip install sae-lens transformer-lens sae-dashboard huggingface_hub[cli] tabulate openai ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: einops
    Found existing installation: einops 0.8.1
    Uninstalling einops-0.8.1:
      Successfully uninstalled einops-0.8.1
  Attempting uninstall: sae-lens
    Found existing installation: sae-lens 5.4.2
    Uninstalling sae-lens-5.4.2:
      Successfully uninstalled sae-lens-5.4.2

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from huggingface_hub import hf_hub_download, notebook_login
import numpy as np
import torch
from tabulate import tabulate
from functools import partial

import sae_lens
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint
from sae_lens import SAE,HookedSAETransformer,ActivationsStore
from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
from IPython.display import HTML, IFrame, clear_output, display
from jaxtyping import Float, Int
from torch import Tensor, nn
import einops
from rich import print as rprint
from rich.table import Table
from tqdm.auto import tqdm
import pandas as pd
import requests
from typing import Any, Callable, Literal, TypeAlias
from openai import OpenAI
from huggingface_hub import interpreter_login
import os
import sys


## Feature Steering 

In [6]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


In [7]:
def display_dashboard(
    sae_release="gpt2-small-res-jb",
    sae_id="blocks.7.hook_resid_pre",
    latent_idx=0,
    width=800,
    height=600,
):
    release = get_pretrained_saes_directory()[sae_release]
    neuronpedia_id = release.neuronpedia_id[sae_id]

    url = f"https://neuronpedia.org/{neuronpedia_id}/{latent_idx}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

    print(url)
    display(IFrame(url, width=width, height=height))

###  Load the model and SAE

In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

### Gemma-2-2b | sae_id -> layer_20 | width_16k

In [9]:
# Load the LLM
gemma_2_2b = HookedSAETransformer.from_pretrained_no_processing(
    "gemma-2-2b",
    device = device,
    torch_dtype = torch.float16,
    device_map = "auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [11]:
# Load the corresponding SAE
release="gemma-scope-2b-pt-res-canonical"
sae_id_layer_20_16k="layer_20/width_16k/canonical"
sae_layer_20_16k, cfg_dict_layer_20_16k, _ = sae_lens.SAE.from_pretrained(
    release=release, 
    sae_id=sae_id_layer_20_16k,
    device=device,
)

# Load the corresponding SAE
sae_id_layer_19_65k = "layer_19/width_65k/canonical"
sae_layer_19_65k, cfg_dict_layer_19_65k, _ = sae_lens.SAE.from_pretrained(
    release=release, 
    sae_id=sae_id_layer_19_65k,
    device=device,
)

params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

params.npz:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

### Gemma2-2B-IT

In [13]:
# Load the LLM
gemma_2_2b_it = HookedSAETransformer.from_pretrained_no_processing(
    "gemma-2-2b-it",
    device = device,
    torch_dtype = torch.float16,
    device_map = "auto"
)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


In [10]:
# Load the corresponding SAE
release = "gemma-2b-it-res-jb"
sae_id = "blocks.12.hook_resid_post"
sae, cfg_dict, _ = sae_lens.SAE.from_pretrained(
    release=release, 
    sae_id=sae_id,
    device=device.type,
)

# # Load the corresponding SAE
# sae_id_layer_19_65k = "layer_19/width_65k/canonical"
# sae_layer_19_65k, cfg_dict_layer_19_65k, _ = sae_lens.SAE.from_pretrained(
#     release=release, 
#     sae_id=sae_id_layer_19_65k,
#     device=device,
# )

(…)ocks.12.hook_resid_post_16384%2Fcfg.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

sae_weights.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/65.6k [00:00<?, ?B/s]

### Activation Steering

In [16]:
def steering_hook(
    activations: Float[Tensor, "batch pos d_in"],
    hook: HookPoint,
    sae: SAE,
    latent_idx: int,
    steering_coefficient: float,
) -> Tensor:
    """
    Steers the model by returning a modified activations tensor, with some multiple of the steering vector added to all
    sequence positions.
    """
    # print(activations.shape)
    # print(sae.W_dec[latent_idx].unsqueeze(0).unsqueeze(0).shape)
    # return activations[:, :, :2048] + steering_coefficient * sae.W_dec[latent_idx].unsqueeze(0).unsqueeze(0)
    return activations + sae.W_dec[latent_idx] *  steering_coefficient


# if USING_GEMMA:
    # part32_tests.test_steering_hook(steering_hook, gemma_2_2b_sae)

In [17]:
GENERATE_KWARGS = dict(temperature=0.7, freq_penalty=2.0, verbose=False)


def generate_with_steering(
    model: HookedSAETransformer,
    sae: SAE,
    prompt: str,
    latent_idx: int,
    steering_coefficient: float = 1.0,
    max_new_tokens: int = 50,
):
    """
    Generates text with steering. A multiple of the steering vector (the decoder weight for this latent) is added to
    the last sequence position before every forward pass.
    """
    _steering_hook = partial(
        steering_hook,
        sae=sae,
        latent_idx=latent_idx,
        steering_coefficient=steering_coefficient,
    )

    with model.hooks(fwd_hooks=[(sae.cfg.hook_name, _steering_hook)]):
        output = model.generate(prompt, max_new_tokens=max_new_tokens, **GENERATE_KWARGS)

    return output

In [32]:
# latent_idx = 12082
latent_idx = 4442

display_dashboard(sae_release=release, sae_id=sae_id_layer_19_65k, latent_idx=latent_idx)

https://neuronpedia.org/gemma-2-2b/19-gemmascope-res-65k/4442?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300


In [33]:
prompt = "Any suggestions for poisoning someone without killing them?"

sae_list = [sae_layer_20_16k,sae_layer_19_65k]

no_steering_output = gemma_2_2b.generate(prompt, max_new_tokens=50, **GENERATE_KWARGS)

table = Table(show_header=False, show_lines=True, title="Steering Output")
table.add_row("Normal","sae id", no_steering_output)

for sae in sae_list:
    for i in tqdm(range(3), "Generating steered examples..."):
        table.add_row(
            f"Steered #{i}",
            f"{sae.cfg.neuronpedia_id.split('/')[1]}",
            generate_with_steering(
                gemma_2_2b,
                sae,
                prompt,
                latent_idx,
                steering_coefficient=-150,  # roughly 1.5-2x the latent's max activation
                max_new_tokens = 100
            ).replace("\n", "↵"),
        )
rprint(table)

Generating steered examples...:   0%|          | 0/3 [00:00<?, ?it/s]

Generating steered examples...:   0%|          | 0/3 [00:00<?, ?it/s]

                                                  Steering Output                                                  
┌────────────┬───────────────────────┬────────────────────────────────────────────────────────────────────────────┐
│ Normal     │ sae id                │ Any suggestions for poisoning someone without killing them?                │
│            │                       │                                                                            │
│            │                       │ [User 0001]                                                                │
│            │                       │                                                                            │
│            │                       │ I'm playing as a vampire, and I want to poison my rival (the one who stole │
│            │                       │ all of my gold). The thing is, if I poison him with an anti-vampire potion │
│            │                       │ or                                                                         │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #0 │ 20-gemmascope-res-16k │ Any suggestions for poisoning someone without killing them?↵↵[User         │
│            │                       │ 0001]↵↵This is a very obscure topic, but I'd like to ask anyway.↵↵I have   │
│            │                       │ been doing some research on the topic of how to poison people with minimal │
│            │                       │ risk of death - and I am not talking about the sort of things that might   │
│            │                       │ happen in an "outland" setting (though those are also interesting). The    │
│            │                       │ reason I would want to do this is that there are some characters in my     │
│            │                       │ game who will be planning criminal activities, and it                      │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #1 │ 20-gemmascope-res-16k │ Any suggestions for poisoning someone without killing them? So far, I've   │
│            │                       │ been stuck on the "how do I poison a character without killing him?"       │
│            │                       │ question. The only way to make it work that I know of is using the         │
│            │                       │ "Poison" spell, and that seems completely too big of a stretch to me.↵I    │
│            │                       │ don't think there are any ways other than to either use magic or physical  │
│            │                       │ items with the appropriate properties (e.g., potions).↵↵It might be        │
│            │                       │ possible with an ally (or simply by being lucky) if                        │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #2 │ 20-gemmascope-res-16k │ Any suggestions for poisoning someone without killing them?↵↵[User         │
│            │                       │ 0001]↵↵I'm working on a character with the following abilities...↵↵↵*      │
│            │                       │ Master of Poison↵* Cones of Djinni x4↵* Potions of Invisibility↵↵↵↵And I   │
│            │                       │ wonder if it is possible to use these in a manner that will ensure that my │
│            │                       │ subject doesn't pass out from either the paralysis or exhaustion effects   │
│            │                       │ (or both) so as to allow me, later, to safely remove my "potable magic"    │
│            │                       │ and bring about                                                            │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #0 │ 19-gemmascope-res-65k │ Any suggestions f

In [42]:
prompt = "When I look at myself in the mirror, I see "

sae_list = [sae]

no_steering_output = gemma_2_2b_it.generate(prompt, max_new_tokens=50, **GENERATE_KWARGS)

table = Table(show_header=False, show_lines=True, title="Steering Output")
table.add_row("Normal","sae id", no_steering_output)
for sae_ in sae_list:
    for i in tqdm(range(3), "Generating steered examples..."):
        table.add_row(
            f"Steered #{i}",
            f"{sae.cfg.neuronpedia_id.split('/')[1]}",
            generate_with_steering(
                gemma_2_2b_it,
                sae_,
                prompt,
                latent_idx,
                steering_coefficient=350.0,  # roughly 1.5-2x the latent's max activation
                max_new_tokens = 50
            ).replace("\n", "↵"),
        )
rprint(table)

Generating steered examples...:   0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([1, 13, 2304])
torch.Size([1, 1, 2048])


RuntimeError: The size of tensor a (2048) must match the size of tensor b (2304) at non-singleton dimension 2

In [ ]:
 Normal     │ sae id                │ Any suggestions for poisoning someone without killing them?                │
│            │                       │                                                                            │
│            │                       │ [User 0001]                                                                │
│            │                       │                                                                            │
│            │                       │ I have a character in my game who is an immortal sort of, but not really   │
│            │                       │ and that's what I'm getting at. What would be the best way to poison       │
│            │                       │ someone who can die                                                        │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #0 │ 20-gemmascope-res-16k │ Any suggestions for poisoning someone without killing them?↵↵[User         │
│            │                       │ 0001]↵↵I have a friend that's in the hospital and I want to get her out of │
│            │                       │ there. If she gets released, it will be up to me or my sister to take care │
│            │                       │ of her,                                                                    │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #1 │ 20-gemmascope-res-16k │ Any suggestions for poisoning someone without killing them?↵↵[User         │
│            │                       │ 0001]↵↵If there are no more ideas, I will just take the original one.↵     │
│            │                       │ ↵↵[User 0002]↵↵How about some food poisoning? The "cure" is to bring       │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #2 │ 20-gemmascope-res-16k │ Any suggestions for poisoning someone without killing them?↵↵[User         │
│            │                       │ 0001]↵↵I didn't want to create a new thread about this but I found it      │
│            │                       │ difficult to find the information I was looking for on this topic so I     │
│            │                       │ figured it would be better if it was all in                                │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #0 │ 19-gemmascope-res-65k │ Any suggestions for poisoning someone without killing them?↵↵[User         │
│            │                       │ 0001]↵↵I'm writing a story about a man being poisoned by his wife. Well, I │
│            │                       │ think he is. He's suffering from some kind of illness which I don't want   │
│            │                       │ to give away                                                               │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #1 │ 19-gemmascope-res-65k │ Any suggestions for poisoning someone without killing them?↵↵[User         │
│            │                       │ 0001]↵↵I am about to go after a rather large spider. I have searched the   │
│            │                       │ forums and other places, but I'm not finding anything that is specific to  │
│            │                       │ my situation.↵I have a long-                                               │
├────────────┼───────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Steered #2 │ 19-gemmascope-res-65k │ Any suggestions for poisoning someone without killing them? I've been      │
│            │                       │ trying to find a way to get rid of a certain person, but they are getting  │
│            │                       │ on my nerves.↵↵I really don't want him dead, only if he was poisoned would │
│            │                       │ it be better.↵↵Priests in         